In [25]:
import warnings
warnings.filterwarnings("ignore")

import logging
import pandas as pd
import os
import gzip

from datetime import datetime
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams

# importo los .py
import sys
sys.path.append("auxiliares")
import metricas

In [26]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [27]:
mes_train_tope = 201902
mes_holdout = 201904

In [28]:
arch_prod_ids_prediccion = "datasets/productos_a_predecir_201902.csv"
arch_sellout_productos = "datasets/emp3_sellout_producto_base.csv"

In [29]:
df_sellout = pd.read_csv(arch_sellout_productos)

In [30]:
df_sellout.head()

,product_id,periodo,product_id_tn,cero_ventas,cust_request_qty,cust_request_tn,mes,producto_estrella,meses_historia_prod,cat1,cat2,cat3,sku_size,temp_media,temp_min_media,temp_max_media,IPC,promedio_mens_dolar_venta,catastrofe
0,20001,201701,934.77222,0,479.0,937.72717,1,1.0,0,HC,ROPA LAVADO,Liquido,3000.0,25.65,20.9,30.4,1.6,15.91,False
1,20001,201702,798.01620,0,432.0,833.72187,2,1.0,1,HC,ROPA LAVADO,Liquido,3000.0,25.35,20.6,30.1,2.1,15.59,False
2,20001,201703,1303.35771,0,509.0,1330.74697,3,1.0,2,HC,ROPA LAVADO,Liquido,3000.0,22.45,18.0,26.9,2.4,15.52,False
3,20001,201704,1069.96130,0,279.0,1132.94430,4,1.0,3,HC,ROPA LAVADO,Liquido,3000.0,18.90,14.3,23.5,2.7,15.36,False
4,20001,201705,1502.20132,0,701.0,1550.68936,5,1.0,4,HC,ROPA LAVADO,Liquido,3000.0,15.90,12.3,19.5,1.4,15.72,False


In [31]:
df_sellout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35888 entries, 0 to 35887
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   product_id                 35888 non-null  int64  
 1   periodo                    35888 non-null  int64  
 2   product_id_tn              35888 non-null  float64
 3   cero_ventas                35888 non-null  int64  
 4   cust_request_qty           35888 non-null  float64
 5   cust_request_tn            35888 non-null  float64
 6   mes                        35888 non-null  int64  
 7   producto_estrella          35888 non-null  float64
 8   meses_historia_prod        35888 non-null  int64  
 9   cat1                       35888 non-null  object 
 10  cat2                       35888 non-null  object 
 11  cat3                       35888 non-null  object 
 12  sku_size                   35888 non-null  float64
 13  temp_media                 35888 non-null  flo

In [32]:
df_sellout.cat1.value_counts()

PC             19181
HC              8115
FOODS           7160
desconocida     1432
Name: cat1, dtype: int64

In [33]:
# Selecciono las columnas iniciales
df_sellout = df_sellout.rename(columns={"product_id_tn":"tn"})

df_sellout = df_sellout[["periodo","product_id","tn","cat1"]]

Verificación de producto estrella --> ok

## Split train y test

In [34]:
df_train = df_sellout[df_sellout.periodo <= mes_train_tope]
df_test = df_sellout[df_sellout.periodo == mes_holdout]

In [35]:
df_train

,periodo,product_id,tn,cat1
0,201701,20001,934.77222,HC
1,201702,20001,798.01620,HC
2,201703,20001,1303.35771,HC
3,201704,20001,1069.96130,HC
4,201705,20001,1502.20132,HC
...,...,...,...,...
35873,201810,21299,0.00000,desconocida
35874,201811,21299,0.00000,desconocida
35875,201812,21299,0.00000,desconocida
35876,201901,21299,0.00000,desconocida


In [36]:
print(len(df_test))
df_test

1105


,periodo,product_id,tn,cat1
27,201904,20001,1647.63848,HC
63,201904,20002,1287.62346,HC
99,201904,20003,565.33774,FOODS
135,201904,20004,466.70901,FOODS
171,201904,20005,624.99880,FOODS
...,...,...,...,...
35756,201904,21295,0.00000,desconocida
35785,201904,21296,0.00000,desconocida
35821,201904,21297,0.00000,desconocida
35850,201904,21298,0.00000,desconocida


In [37]:
df_prods_prediccion=pd.read_csv(arch_prod_ids_prediccion)
df_prods_prediccion.head()

,product_id
0,20480
1,20481
2,20482
3,20483
4,20484


In [38]:
df_test = df_test[df_test.product_id.isin(df_prods_prediccion.product_id)]
len(df_test)

868

In [39]:
# Como puede haber productos que no estén en algunos períodos (que se usan en modelos naif), calculo promedio para completar
df_completar = df_train[["product_id","tn"]].groupby("product_id").mean().reset_index()
df_completar

,product_id,tn
0,20001,1362.248849
1,20002,943.915757
2,20003,934.905062
3,20004,677.100396
4,20005,613.286967
...,...,...
1050,21295,0.000269
1051,21296,0.000343
1052,21297,0.000223
1053,21298,0.000302


## Predicción Estacional

In [40]:
df_pred_estac = df_train[df_train.periodo==201804]
df_pred_estac = df_pred_estac[["product_id","tn"]]
df_pred_estac.head()

,product_id,tn
15,20001,1251.28462
51,20002,999.20934
87,20003,765.47838
123,20004,611.51237
159,20005,496.41774


In [41]:
# Left join de test con prediccion estacional
df_res_estac = pd.merge(df_test,df_pred_estac,on="product_id",how="left",suffixes=["_real","_estac"])

# Left join con prediccion promedio (para completar)
df_res_estac = pd.merge(df_res_estac,df_completar,on="product_id",how="left")
df_res_estac.rename(columns={'tn': 'tn_prom'}, inplace=True)

# Calculo toneladas predichas usando los valores anteriores
df_res_estac["tn_pred"]=np.where(df_res_estac.tn_estac.isnull(),df_res_estac.tn_prom,df_res_estac.tn_estac)

df_res_estac = df_res_estac[["product_id","cat1","tn_real","tn_pred"]]
df_res_estac.head()

,product_id,cat1,tn_real,tn_pred
0,20001,HC,1647.63848,1251.28462
1,20002,HC,1287.62346,999.20934
2,20003,FOODS,565.33774,765.47838
3,20004,FOODS,466.70901,611.51237
4,20005,FOODS,624.99880,496.41774


In [42]:
df_res_estac.isna().sum()

product_id    0
cat1          0
tn_real       0
tn_pred       0
dtype: int64

In [43]:
metricas.print_error_rate_total_y_cat(df_res_estac)

**Error Rate Total:** 35.09

**Error Rate por Categoría HC**: 33.75

**Error Rate por Categoría FOODS**: 32.1

**Error Rate por Categoría PC**: 42.78

**Error Rate por Categoría desconocida**: inf

## Predicción Menos 2

In [44]:
# Naif -2
df_pred_menos2 = df_train[df_train.periodo==201902]
df_pred_menos2 = df_pred_menos2[["product_id","tn"]]
df_pred_menos2.head()

,product_id,tn
25,20001,1259.09363
61,20002,1043.01349
97,20003,758.32657
133,20004,441.70332
169,20005,409.89950


In [45]:
# Left join de test con prediccion menos 2
df_res_menos2 = pd.merge(df_test,df_pred_menos2,on="product_id",how="left",suffixes=["_real","_menos2"])

# Left join con prediccion promedio (para completar)
df_res_menos2 = pd.merge(df_res_menos2,df_completar,on="product_id",how="left")
df_res_menos2.rename(columns={'tn': 'tn_prom'}, inplace=True)

# Calculo toneladas predichas usando los valores anteriores
df_res_menos2["tn_pred"]=np.where(df_res_menos2.tn_menos2.isnull(),df_res_menos2.tn_prom,df_res_menos2.tn_menos2)

df_res_menos2 = df_res_menos2[["product_id","cat1","tn_real","tn_pred"]]
df_res_menos2.head()

,product_id,cat1,tn_real,tn_pred
0,20001,HC,1647.63848,1259.09363
1,20002,HC,1287.62346,1043.01349
2,20003,FOODS,565.33774,758.32657
3,20004,FOODS,466.70901,441.70332
4,20005,FOODS,624.99880,409.89950


In [46]:
df_res_menos2[df_res_menos2.product_id==20001]

,product_id,cat1,tn_real,tn_pred
0,20001,HC,1647.63848,1259.09363


In [47]:
df_res_estac[df_res_estac.product_id==20001]

,product_id,cat1,tn_real,tn_pred
0,20001,HC,1647.63848,1251.28462


In [48]:
metricas.print_error_rate_total_y_cat(df_res_menos2)

**Error Rate Total:** 29.0

**Error Rate por Categoría HC**: 27.98

**Error Rate por Categoría FOODS**: 27.45

**Error Rate por Categoría PC**: 34.13

**Error Rate por Categoría desconocida**: inf